In [ ]:
# Translating image coordinates to pitch coordinates with homography

## Calculating 4 corners of the pitch

- Assumption 1: We have 4 corners of a sub-area, e.g. TLC-TRC-TLF-TRF
- Assumption 2: Areas on the video are parallel, i.e. the camera stays in the centre

## Calculate the top vertex of the triangle containing the projected view


In [6]:
from typing import Tuple, Union
import numpy as np

def calculate_intersection_of_lines(segment1: np.ndarray, segment2: np.ndarray) -> Union[Tuple[float, float], None]:
    """
    Calculate intersection of lines defined by segments.
    Reference: https://en.wikipedia.org/wiki/Line%E2%80%93line_intersection#Given_two_points_on_each_line
    Return coordinates of a point of intersection, or None if lines defined by segments are parallel.
    """
    if segment1.shape != (4,):
        raise ValueError("segment1 must be a vector of size 4")
    if segment2.shape != (4,):
        raise ValueError("segment2 must be a vector of size 4")
    x1, y1, x2, y2 = segment1[0], segment1[1], segment1[2], segment1[3]
    x3, y3, x4, y4 = segment2[0], segment2[1], segment2[2], segment2[3]

    def quarter_determinant(arr: np.ndarray) -> float:
        """Given a matrix of size 4x4, calculate a determinant of 4 determinants from each quarter
        """
        if arr.shape != (4,4):
            raise ValueError("arr must be a matrix of size 4x4")
        a1 = np.linalg.det(arr[0:2,0:2])
        a2 = np.linalg.det(arr[0:2,2:4])
        a3 = np.linalg.det(arr[2:4, 0:2])
        a4 = np.linalg.det(arr[2:4, 2:4])
        return np.linalg.det(np.array([[a1, a2], [a3, a4]]))

    px_num = np.array([
        [x1, y1, x1, 1],
        [x2, y2, x2, 1],
        [x3, y3, x3, 1],
        [x4, y4, x4, 1]])
    # px_div = np.array([
    #     [x1, 1, y1, 1],
    #     [x2, 1, y2, 1],
    #     [x3, 1, y3, 1],
    #     [x4, 1, y4, 1]])
    py_num = np.array([
        [x1, y1, y1, 1],
        [x2, y2, y2, 1],
        [x3, y3, y3, 1],
        [x4, y4, y4, 1]])
    pxy_div = np.array([
        [x1, 1, y1, 1],
        [x2, 1, y2, 1],
        [x3, 1, y3, 1],
        [x4, 1, y4, 1]])
    
    div_det = quarter_determinant(pxy_div)
    if abs(div_det) < 0.00001:
        # segment1 and segment2 seem to be parallel
        return None
    px = quarter_determinant(px_num)/div_det
    py = quarter_determinant(py_num)/div_det
    return px, py


Example:
31 0.699563 0.102965 0.0109856 0.0191795 0.914857  #TRC
31 0.342411 0.104897 0.00925484 0.0163508 0.699574 #TLC
31 0.731543 0.135071 0.0102163 0.0179255 0.637236  #TRF
31 0.310968 0.135658 0.00958964 0.0163189 0.448016  #TLF

In [7]:
segment1 = np.array([0.310968, 0.135658, 0.342411, 0.104897])  #TLF-TLC
segment2 = np.array([0.731543, 0.135071, 0.699563, 0.102965])  #TRF-TRC
print(calculate_intersection_of_lines(segment1, segment2))

(0.5242705938932326, -0.07301805160925191)


# Find homography

In [39]:
import cv2

# Define the corresponding points in the two images
points1 = np.float32([[240, 17], [1, 20], [20, 30], [30, 30]])
print(points1.shape)
points2 = np.float32([[120, 80], [120, 50], [200, 50], [200, 80]])

# Calculate the homography matrix
H, mask = cv2.findHomography(points1, points2)

print(H)
pt = np.dot(H, np.float32([240, 17, 1]))
pt = pt/pt[2]
print(pt)

(4, 2)
[[ 4.44144122e-03 -3.78778794e+00  1.17167545e+02]
 [ 5.61176098e-02 -1.74173122e+00  5.20352682e+01]
 [ 2.22072061e-05 -3.27443489e-02  1.00000000e+00]]
[120.  80.   1.]


In [40]:
import cv2
#pitch_points = [[0.310968, 0.135658], [0.342411, 0.104897], [0.731543, 0.135071, 0.699563, 0.102965]]
#src_points = np.array(cv2.Point2f
src_points = np.array([[0.342411, 0.104897], [0.699563, 0.102965], [0.310968, 0.135658], [0.731543, 0.135071]])
assert src_points.shape == (4,2)
dst_points = np.array([[0,0],[37,0],[0,18],[37,18]])
H, mask = cv2.findHomography(src_points, dst_points)
print(H)

[[ 1.18920673e+01 -1.53363315e+02  1.20153769e+01]
 [-4.38007353e-01 -8.09704958e+01  8.64354063e+00]
 [-2.96646610e-02 -8.31785968e+00  1.00000000e+00]]


In [48]:
def convert_h(H, vec):
    pt = np.dot(H, vec)
    return pt/pt[2]

vectors = np.float32([
    [0.342411, 0.104897, 1],
    [0.699563, 0.102965, 1],
    [0.310968, 0.135658, 1],
    [0.731543, 0.135071, 1]])
for v in vectors[:,0:3]:
    print(f"v: {v}; converted: {convert_h(H, v)}")
#convert_h(H, vec)

v: [0.342411 0.104897 1.      ]; converted: [0. 0. 1.]
v: [0.699563 0.102965 1.      ]; converted: [3.7000000e+01 1.4465529e-14 1.0000000e+00]
v: [0.310968 0.135658 1.      ]; converted: [37. 18.  1.]
v: [0.731543 0.135071 1.      ]; converted: [-0. 18.  1.]


In [34]:
import cv2
import numpy as np

# Define the homography matrix H
H = np.array([[1.4, 0.05, -100], [0.05, 1.5, -50], [0.001, 0.001, 1]])

# Define a point in the first image
point1 = np.array([[300], [200], [1]])
#point1 = np.array([300, 200, 1])

# Use the homography matrix to translate the point to the second image
point2 = np.dot(H, point1)
print(point2)
# Convert the point back to Cartesian coordinates
point2 = point2 / point2[2]

print(point2)

[[330. ]
 [265. ]
 [  1.5]]
[[220.        ]
 [176.66666667]
 [  1.        ]]
